# Entendiendo modelos
- Entender cómo se utilizan los modelos de ML para hacer el forecast
- Entender qué features ve el modelo para hacer el forecast

In [1]:
import random
import tempfile
from pathlib import Path

import pandas as pd
from datasetsforecast.m4 import M4
from utilsforecast.plotting import plot_series

from sklearn.linear_model import LinearRegression
from mlforecast import MLForecast

import numpy as np

### 0. Data

In [2]:
await M4.async_download('data', group='Hourly')
df, *_ = M4.load('data', 'Hourly')
uids = df['unique_id'].unique()
random.seed(0)
sample_uids = random.choices(uids, k=4)
df = df[df['unique_id'].isin(sample_uids)].reset_index(drop=True)
df['ds'] = df['ds'].astype('int64')

In [3]:
df.head()

,unique_id,ds,y
0,H196,1,11.8
1,H196,2,11.4
2,H196,3,11.1
3,H196,4,10.8
4,H196,5,10.6


### 1. Entrenar modelos
- Entrenar modelo que ve 4 series de tiempo
- Entrenar modelo que ve 1 serie de tiempo

In [4]:
# obtener dataset con 4 series
df_4series = df.copy()

In [5]:
# crear modelo que ve 4 series
fcst_4series = MLForecast(
    models=[LinearRegression()],
    freq=1,
    lags=[1, 2, 3],
)

In [6]:
fcst_4series.fit(df_4series)

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [7]:
# obtener el ultimo valor real
df_4series.groupby('unique_id').tail(1)

,unique_id,ds,y
1007,H196,1008,16.8
2015,H256,1008,13.4
3023,H381,1008,207.0
4031,H413,1008,34.0


In [8]:
# fcst un horizonte de tiempo
fcst_4series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,18.830526
1,H256,1009,15.671004
2,H381,1009,209.659443
3,H413,1009,32.658524


In [9]:
# obtener la ultima instancia X con los datos transformados. De cada serie
fcst_4series.preprocess(df_4series).groupby('unique_id').tail(1)

,unique_id,ds,y,lag1,lag2,lag3
1007,H196,1008,16.8,17.3,17.8,18.6
2015,H256,1008,13.4,13.8,14.3,14.8
3023,H381,1008,207.0,169.0,148.0,176.0
4031,H413,1008,34.0,41.0,47.0,88.0


In [10]:
# guardar modelo

# path folder: # solo es necesario indicar folder donde guardar. Internamente se guardan 2 pkl (modelos de ml y arquitectura de nixtla)
folder_path_4series = 'models/fcst_4series' 
# save
fcst_4series.save(folder_path_4series)

#### 1.2 entrenar modelo que ve solo 1 serie

In [11]:
# crear df que tiene solo una serie
df_1series = df[df['unique_id'] == 'H196']
df_1series.head(3)

,unique_id,ds,y
0,H196,1,11.8
1,H196,2,11.4
2,H196,3,11.1


In [12]:
# crear modelo - misma arquitectura que modelo que ve 4 series
fcst_1series = MLForecast(
    models=[LinearRegression()],
    freq=1,
    lags=[1, 2, 3],
)
fcst_1series

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [13]:
fcst_1series.fit(df_1series)

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [14]:
# fcst un horizonte de tiempo
fcst_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


In [15]:
# obtener la ultima instancia con los datos transformados
fcst_1series.preprocess(df_1series).groupby('unique_id').tail(1)

,unique_id,ds,y,lag1,lag2,lag3
1007,H196,1008,16.8,17.3,17.8,18.6


In [16]:
# guardar modelo
folder_path_1series = 'models/fcst_1series' 
fcst_1series.save(folder_path_1series)

#### 1.3 Modelo Random Forecast sklearn 4 series de tiempo

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [18]:
# obtener dataset con 4 series
df_4series = df.copy()

In [19]:
# crear modelo que ve 4 series
fcst_4series_rf = MLForecast(
    models=[RandomForestRegressor(random_state = 42, n_estimators = 10)],
    freq=1,
    lags=[1, 2, 3],
)

In [20]:
fcst_4series_rf.fit(df_4series)

MLForecast(models=[RandomForestRegressor], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [21]:
# obtener el ultimo valor real
df_4series.groupby('unique_id').tail(1)

,unique_id,ds,y
1007,H196,1008,16.8
2015,H256,1008,13.4
3023,H381,1008,207.0
4031,H413,1008,34.0


In [22]:
# fcst un horizonte de tiempo
fcst_4series_rf.predict(1)

,unique_id,ds,RandomForestRegressor
0,H196,1009,16.340000
1,H256,1009,13.038139
2,H381,1009,185.400000
3,H413,1009,30.400000


In [23]:
# guardar modelo
folder_path_4series_rf = 'models/fcst_4series_rf' 
fcst_4series_rf.save(folder_path_4series_rf)

### 2. Obtener la clase con el modelo y hacer predicción con el modelo individual

#### 2.1 Leer Modelo sklearn entrenado con una serie de tiempo - LECTURA DIRECTA

In [24]:
# leer artefacto con el modelo
dict_model = pd.read_pickle('models/fcst_1series/models.pkl')
model_lr_1series = dict_model['LinearRegression']
model_lr_1series

LinearRegression()

In [25]:
# obtener istancia - ultima observacion en los datos
df_1series.tail(1)

,unique_id,ds,y
1007,H196,1008,16.8


In [26]:
# obtener instancia con el procesamiento
instance_1series = fcst_1series.preprocess(df_1series).tail(1)
instance_1series

,unique_id,ds,y,lag1,lag2,lag3
1007,H196,1008,16.8,17.3,17.8,18.6


In [27]:
# hacer predicción con el artefacto leido sin considerar clase se nixtla - ml forecast
# IMPORTANTE ESTO DA ERROR PORQUE EL MODELO NO VE TODAS LAS FEATURES DEL DATAFRAME INPUT
# model.predict(instance_aux)

## IMPORTANTE:
- Para entrenar el modelo, SOLO ALGUNAS alguna variable se debe de pasar
- Esto porque el modelo no ve el target (y). Sino que ve alguna variable lageada
- El modelo **NO VE LAS SIGUIENTES VARIABLES: ds, unique_id, y**

CONCLUSIÓN IMPORTANTE:
- Cuando un modelo se entrena con multiples series de tiempo, se pasa el índice de la serie, pero este NO es utilizado para entrenar el modelo. Los datos de cada serie se pasan como una única feature, entonces el modelo tiene que se capaz de identificar los patrones y poder hacer el forecast

In [28]:
# generar la instancia haciendo el drop de las columnas que no se utilizan para entrenar el modelo
list_feature_not_used = ['unique_id', 'ds', 'y']
instance_1series = instance_1series.drop(columns = list_feature_not_used)
instance_1series

,lag1,lag2,lag3
1007,17.3,17.8,18.6


In [29]:
model_lr_1series.predict(instance_1series)

array([16.93344762])

#### 2.2 Generar observación para forecast 1 horizonte. Modelo directo
En el paso anterior se hizo el predict de la última observación. Ahora generar el input para el forecast h+1

In [30]:
df_1series.tail()

,unique_id,ds,y
1003,H196,1004,19.7
1004,H196,1005,18.6
1005,H196,1006,17.8
1006,H196,1007,17.3
1007,H196,1008,16.8


In [31]:
# generar manualmente el input lag 1, lag 2, lag 3
array_values_instance_fcst = df_1series.tail(3)['y'].values
array_values_instance_fcst = np.flip(array_values_instance_fcst)

df_instance_fcst = pd.DataFrame(array_values_instance_fcst.reshape(1, array_values_instance_fcst.shape[0]),
                                columns = ['lag1', 'lag2', 'lag3']
                               )

df_instance_fcst

,lag1,lag2,lag3
0,16.8,17.3,17.8


In [32]:
# luego de generar el input que SERIA PARA FORECAST H+1
# predecir con el modelo leido directamente
model_lr_1series.predict(df_instance_fcst)

array([16.36245009])

#### Luego para predecir las siguientes observaciones, se hace de forma recursiva utilizando las predicciones de las etapas previas

#### 2.3 Leer modelo de nixtla que se entrenó usando una serie de tiempo

In [33]:
# leer modelo de nixtla, que contiene solo el modelo leido de forma DIRECTA en el paso anterior
path_folder_nixtla_1serie = 'models/fcst_1series/'
model_nixtla_1series = MLForecast.load(path_folder_nixtla_1serie)
model_nixtla_1series

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [34]:
# predecir 1 observación futuro
model_nixtla_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


#### 2.3 SE OBSERVA QUE LAS PREDICCIONES PARA FORECAST 1 HORIZONTE FUTURE SON IGUALES ENTRE LECTURA DIRECTA Y LECTURA CON NIXTLA
- Esto es lo que debería de pasar, todo ok
- La siguiente prueba es la última, entrenar modelo que ve más de una serie de tiempo. Validar que se sigue funcionando la lógica de recursividad y por lo tanto reforzar la idea que se pasan todas las series de tiempo juntas

In [35]:
# lectura directa modelo
model_lr_1series.predict(df_instance_fcst)

array([16.36245009])

In [36]:
# lectura modelo nixtla
model_nixtla_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


#### 2.4 Qué pasa si se entrena el modelo utilizando sklearn en lugar de nixtla
Hacer el mismo procesamiento de datos, solo que entrenar directamente en sklarn

In [37]:
# obtener dataframe para entrenamiento
df_X_train = model_nixtla_1series.preprocess(df_1series)
df_X_train = df_X_train.drop(columns = ['unique_id', 'ds', 'y'])
df_X_train.head(3)

,lag1,lag2,lag3
3,11.1,11.4,11.8
4,10.8,11.1,11.4
5,10.6,10.8,11.1


In [38]:
df_y_train = model_nixtla_1series.preprocess(df_1series)[['y']]
df_y_train.head(3)

,y
3,10.8
4,10.6
5,10.3


In [39]:
print('shape')
print('X: ', df_X_train.shape)
print('y: ', df_y_train.shape)

shape
X:  (1005, 3)
y:  (1005, 1)


In [40]:
# entrenar modelo
from sklearn.linear_model import LinearRegression
model_scratch = LinearRegression()
model_scratch.fit(df_X_train, df_y_train)

LinearRegression()

In [41]:
# realizar inferencia forecast h+1 y comparar resultados
model_scratch.predict(df_instance_fcst)

array([[16.36245009]])

### CONCLUSION
- SE PUEDE OBSERVAR QUE LOS RESULTADOS SON LOS MISMOS ENTRENANDO EL MODELO DESDE NIXTLA O ENTRENANDO EL MODELO DIRECTAMENTE DESDE SKLEARN
- LA ÚNICA DIFERENCIA ES QUE NIXTLA PERMITE EL FORECAST DE FORMA RECURSIVA DE FORMA MUCHO MÁS FÁCIL MIENTRAS QUE EN OTROS PACKAGES SE DEBE DE HACER MANUALMENTE

In [42]:
# resultado leer modelo directamente
model_lr_1series.predict(df_instance_fcst)

array([16.36245009])

In [43]:
# resultado leer modelo nixtla
model_nixtla_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


In [44]:
# resultado entrenar modelo
model_scratch.predict(df_instance_fcst)

array([[16.36245009]])

## MULTIPLE SERIES

#### 2.5 Ahora si cargo un modelo entrenado con más series de tiempo

In [45]:
# leer artefacto con el modelo
dict_model = pd.read_pickle('models/fcst_4series/models.pkl')
model_lr_4series = dict_model['LinearRegression']
model_lr_4series

LinearRegression()

In [46]:
df_4series.tail()

,unique_id,ds,y
4027,H413,1004,99.0
4028,H413,1005,88.0
4029,H413,1006,47.0
4030,H413,1007,41.0
4031,H413,1008,34.0


In [47]:
# generar manualmente el input lag 1, lag 2, lag 3. DE LA ÚLTIMA SERIE DE TIEMPO
array_values_instance_fcst = df_4series.tail(3)['y'].values
array_values_instance_fcst = np.flip(array_values_instance_fcst)

df_instance_fcst = pd.DataFrame(array_values_instance_fcst.reshape(1, array_values_instance_fcst.shape[0]),
                                columns = ['lag1', 'lag2', 'lag3']
                               )

df_instance_fcst

,lag1,lag2,lag3
0,34.0,41.0,47.0


In [48]:
# PREDECIR
model_lr_4series.predict(df_instance_fcst)

array([32.6585243])

#### 2.6 Leer modelo de nixtla y hacer forecast 1 horizonte de tiempo

In [49]:
# leer modelo de nixtla, que contiene solo el modelo leido de forma DIRECTA en el paso anterior
path_folder_nixtla_1serie = 'models/fcst_4series/'
model_nixtla_4series = MLForecast.load(path_folder_nixtla_1serie)
model_nixtla_4series

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [50]:
# predecir 1 observación futuro
model_nixtla_4series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,18.830526
1,H256,1009,15.671004
2,H381,1009,209.659443
3,H413,1009,32.658524


#### 2.7 Entrenar modelo pasando todas las series juntas sin identifcar el tipo de serie

In [51]:
# obtener dataframe para entrenamiento
df_X_train = model_nixtla_4series.preprocess(df_4series)
df_X_train = df_X_train.drop(columns = ['unique_id', 'ds', 'y'])
df_X_train.head(3)

,lag1,lag2,lag3
3,11.1,11.4,11.8
4,10.8,11.1,11.4
5,10.6,10.8,11.1


In [52]:
df_y_train = model_nixtla_4series.preprocess(df_4series)[['y']]
df_y_train.head(3)

,y
3,10.8
4,10.6
5,10.3


In [53]:
print('shape')
print('X: ', df_X_train.shape)
print('y: ', df_y_train.shape)

shape
X:  (4020, 3)
y:  (4020, 1)


In [54]:
# entrenar modelo
from sklearn.linear_model import LinearRegression
model_scratch = LinearRegression()
model_scratch.fit(df_X_train, df_y_train)

LinearRegression()

In [55]:
df_instance_fcst # intancia ultima observacion

,lag1,lag2,lag3
0,34.0,41.0,47.0


In [56]:
# realizar inferencia forecast h+1 y comparar resultados
model_scratch.predict(df_instance_fcst)

array([[32.6585243]])

### CONCLUSIÓN FINAL: El valor predicho para la serie es el mismo
- Esto refuerza que las series se pasan al modelo todas juntas, sin hacer diferencia en el tipo de serie y es el modelo el que debe de hacer el forecast reconociendo los patrones en los datos
- No se pasa el ID de la serie. Se pasan todas las series y que el modelo entienda los patrones. Esto podría mejorar si se entrenan 2 modelos y cada uno recibe distintas series. Hay que entrar a probar, quizas series muy correlacionadas entre sí es mejor pasarlas juntas y se obtienen mejores resultados ¿?
- Al modelo se le pasan todos los datos y se entrena para predecir h+1. Luego con recursividad se obtienen prediciones h horizontes a futuro.

In [57]:
# inferencia ENTRENANDO modelo con las 4 series juntas sin identificar el tipo de serie
model_scratch.predict(df_instance_fcst)

array([[32.6585243]])

In [58]:
# inferencia leyendo el modelo directametne
model_lr_4series.predict(df_instance_fcst)

array([32.6585243])

In [59]:
# leyendo modelo nixtla y haciendo forecast
model_nixtla_4series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,18.830526
1,H256,1009,15.671004
2,H381,1009,209.659443
3,H413,1009,32.658524


#### IMPORTANTE:
- Se puede observar que el modelo es entrenado sin pasar el timestamp, y, unique_id

- AL MODELO se le pasan TODOS los datos. Por lo tanto el ID de la serie solo sirve para darle un nombre a la serie en el output, pero el modelo de por si no hace nada
  
- Me imagino que no sabe, solo ve los datos de entrada y decide qué hacer. Al final si los datos son parecidos, los modelos entrenados van a ser los mismos

- Como el modelo es el mismo directamente o a través de nixtla, se pueden hacer todos los análisis que se harían siempre. Nixtla solamente actual para facilitar la recursividad en el forecast

In [60]:
# obtener coeficientes de la serie. Si fuera un arima, el coeficiente no podría ser mayor a 1
model_lr_4series.coef_

array([ 1.21103453, -0.18011296, -0.09359996])

In [61]:
# se puede obtener el modelo directamente desde la clase de nixtla
model_nixtla_4series.models['LinearRegression'].coef_

array([ 1.21103453, -0.18011296, -0.09359996])

In [64]:
model_nixtla_4series.models

{'LinearRegression': LinearRegression()}

In [67]:
# consultar los modelos cuando se utiliza el método directo para hacer forecast H horizontes de tiempo definidos
model_nixtla_4series.models_

{'LinearRegression': LinearRegression()}